In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import babel.dates
from babel import Locale

### Кейс по нахождению наиболее близкой даты звонка при сопоставлении двух выгрузок из разных программ, а также по выявлению звонков нарушающих ФЗ-230 (телефонные звонки могут осуществляться не более одного раза в день, двух раз в неделю и восьми раз в месяц)

## Алтухов Николай 

Пояснение: есть выгрузка из программы, данные в которой создаются автоматически этой программой (при начале вызова сразу проставляется номер телефона и дата вызова). И есть вторая выгрузка, уже из другой программы, данные в которой заполняются вручную сотрудником; плюс у сотрудника может быть другой часовой пояс; соответственно, даты вызова будут отличаться от тех, которые выставляются программой автоматически.

Задача - сопоставить эти две выгрузки, путем присоединения данных по наиболее близкой дате, с целью вытягивания дополнительной информации (например, столбцов с результатом взаимодействия по звонку или статусом договора) из нужной выгрузки
________

Генерация 10 тысяч случайных российских номеров телефона

In [2]:
phone_numbers = []
for _ in range(10000):
    number = "89" + ''.join(random.choice('0123456789') for _ in range(9))
    phone_numbers.append(number)
phone_numbers = np.array(phone_numbers)

Генерация количества звонков по каждому номеру и дат вызова

In [3]:
# Используем распределение Пуассона для генерации случайного количество звонков на номера телефонов
quantities = np.random.poisson(5, 10000)

# Замена всех нулей на единицы, чтоб все 10000 телефонов остались
quantities[quantities == 0] = 1

# Повторяем каждый элемент массива phone_numbers нужное количество раз
phone_numbers_repeated = np.repeat(phone_numbers, quantities)

In [4]:
start_date = datetime(2024, 7, 1, 0, 0, 0)
end_date = datetime(2024, 7, 31, 23, 59, 59)
delta = end_date - start_date
total_sec = int(delta.total_seconds())
random_dates = []
for _ in range(len(phone_numbers_repeated)):
    # выбор случайной секунды в июле 2024
    random_second = random.randint(0, total_sec)
    random_date = start_date + timedelta(seconds=random_second)
    random_dates.append(random_date)

Генерация результатов взаимодействия, которые мы будем подтягивать во второй датафрейм

In [5]:
results = [
    "Платеж подтвержден",
    "Ответ третьих лиц",
    "Должник отказался платить",
    "Просит реструктуризацию",
    "Должник попросил перезвонить позже"
]

random_results = random.choices(results, k=len(random_dates))


In [6]:
df_calls = pd.DataFrame({'Номер телефона': phone_numbers_repeated, 'Дата вызова': random_dates, "Результат взаимодействия":random_results})

In [7]:
df_calls

,Номер телефона,Дата вызова,Результат взаимодействия
0,89516888842,2024-07-18 20:25:04,Должник попросил перезвонить позже
1,89516888842,2024-07-27 18:37:40,Ответ третьих лиц
2,89516888842,2024-07-23 22:49:12,Платеж подтвержден
3,89516888842,2024-07-08 03:09:11,Ответ третьих лиц
4,89516888842,2024-07-26 04:50:29,Просит реструктуризацию
...,...,...,...
50081,89656149765,2024-07-18 19:47:30,Просит реструктуризацию
50082,89656149765,2024-07-25 23:35:28,Просит реструктуризацию
50083,89656149765,2024-07-02 16:54:08,Должник отказался платить
50084,89656149765,2024-07-20 09:02:09,Должник попросил перезвонить позже


Добавление номера недели и дня недели, для дальнейшего анализа по выявлению звонков нарушающих ФЗ-230

In [8]:
# Устанавливаем локаль для русского языка
locale = Locale('ru', 'RU')

# Функция для получения названия дня недели на русском языке
def get_day_name(date, locale):
    return babel.dates.format_datetime(date, 'EEEE', locale=locale)

df_calls['Номер недели'] = df_calls['Дата вызова'].dt.isocalendar().week
df_calls['День недели'] = df_calls['Дата вызова'].apply(lambda x: get_day_name(x, locale))

df_calls

,Номер телефона,Дата вызова,Результат взаимодействия,Номер недели,День недели
0,89516888842,2024-07-18 20:25:04,Должник попросил перезвонить позже,29,четверг
1,89516888842,2024-07-27 18:37:40,Ответ третьих лиц,30,суббота
2,89516888842,2024-07-23 22:49:12,Платеж подтвержден,30,вторник
3,89516888842,2024-07-08 03:09:11,Ответ третьих лиц,28,понедельник
4,89516888842,2024-07-26 04:50:29,Просит реструктуризацию,30,пятница
...,...,...,...,...,...
50081,89656149765,2024-07-18 19:47:30,Просит реструктуризацию,29,четверг
50082,89656149765,2024-07-25 23:35:28,Просит реструктуризацию,30,четверг
50083,89656149765,2024-07-02 16:54:08,Должник отказался платить,27,вторник
50084,89656149765,2024-07-20 09:02:09,Должник попросил перезвонить позже,29,суббота


Теперь нужно сгенерировать выгрузку из программы, которой пользуется сотрудник. У него может быть другой часовой пояс, и проставить дату вызова он может в течение некоторого времени с начала звонка.

Для этого я возьму созданный датафрейм и изменю даты вызова. Даты будут отличаться на случайное число в недискретном диапазоне 0-3 часа, а также у некоторых (10% от общего количества) будет изменен часовой пояс (я буду добавлять +4, -4 часа)

В выгрузке из программы сотрудника будет 5 тысяч звонков, к которым нужно подтянуть данные по Результату взаимодействия

In [9]:
df_to_match = df_calls[['Номер телефона','Дата вызова']].sample(5000).copy()

In [10]:
random_hours = np.random.uniform(0, 3, size=len(df_to_match))
random_timezone = random.choices([0,4,-4], weights=[90, 5, 5], k=len(df_to_match))

random_hours_timedelta = pd.to_timedelta(random_hours, unit='h')
random_timezone_timedelta = pd.to_timedelta(random_timezone, unit='h')
# Применение случайных значений к датам
df_to_match['Дата вызова от сотрудника'] = df_to_match['Дата вызова'] + random_hours_timedelta + random_timezone_timedelta
df_to_match['Дата вызова от сотрудника'] = df_to_match['Дата вызова от сотрудника'].dt.round('s')
df_to_match = df_to_match.rename(columns={'Дата вызова':"old.Дата вызова"})

In [11]:
df_to_match

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника
6999,89667678258,2024-07-26 12:15:45,2024-07-26 12:50:35
21965,89515202060,2024-07-12 16:34:40,2024-07-12 17:14:07
40511,89143475024,2024-07-19 16:44:55,2024-07-19 19:37:23
8546,89034493393,2024-07-04 02:14:42,2024-07-04 03:11:36
49071,89235526821,2024-07-09 20:15:34,2024-07-09 18:02:37
...,...,...,...
45043,89776292327,2024-07-28 21:21:06,2024-07-29 01:39:45
16228,89420792647,2024-07-13 00:46:53,2024-07-12 22:49:23
13906,89543598231,2024-07-19 22:38:18,2024-07-20 02:55:43
39532,89630661888,2024-07-15 05:30:44,2024-07-15 08:13:31


In [12]:
df_to_match = df_to_match.sort_values(by=['Номер телефона', 'Дата вызова от сотрудника'])

Перемешаем строки выгрузки звонков первого датафрейма, чтоб немного усложнить задачу

In [13]:
df_calls_shuffled = df_calls.sample(frac=1).reset_index(drop=True)

Объединим датафрейм выгрузки сотрудника с перемешанным первым датафреймом, путем соединения LEFT JOIN

In [14]:
merged_df = df_to_match.merge(df_calls_shuffled, on='Номер телефона', how='left')

Вычисление разницы между датами

In [15]:
merged_df

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели
0,89000025589,2024-07-01 10:05:50,2024-07-01 11:59:46,2024-07-23 06:18:18,Платеж подтвержден,30,вторник
1,89000025589,2024-07-01 10:05:50,2024-07-01 11:59:46,2024-07-01 10:05:50,Ответ третьих лиц,27,понедельник
2,89000025589,2024-07-23 06:18:18,2024-07-23 07:36:36,2024-07-23 06:18:18,Платеж подтвержден,30,вторник
3,89000025589,2024-07-23 06:18:18,2024-07-23 07:36:36,2024-07-01 10:05:50,Ответ третьих лиц,27,понедельник
4,89000685220,2024-07-07 09:39:44,2024-07-07 11:36:04,2024-07-25 02:10:25,Должник отказался платить,30,четверг
...,...,...,...,...,...,...,...
30266,89999515783,2024-07-14 08:20:34,2024-07-14 11:04:27,2024-07-20 21:26:21,Должник отказался платить,29,суббота
30267,89999515783,2024-07-14 08:20:34,2024-07-14 11:04:27,2024-07-09 02:59:35,Просит реструктуризацию,28,вторник
30268,89999515783,2024-07-14 08:20:34,2024-07-14 11:04:27,2024-07-14 08:20:34,Ответ третьих лиц,28,воскресенье
30269,89999515783,2024-07-14 08:20:34,2024-07-14 11:04:27,2024-07-07 02:12:17,Платеж подтвержден,27,воскресенье


In [396]:
merged_df['date_diff'] = abs(merged_df['Дата вызова от сотрудника'] - merged_df['Дата вызова'])

In [397]:
merged_df

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
0,89000350892,2024-07-09 04:11:37,2024-07-09 06:59:19,2024-07-29 22:37:31,Просит реструктуризацию,31,понедельник,20 days 15:38:12
1,89000350892,2024-07-09 04:11:37,2024-07-09 06:59:19,2024-07-11 15:09:15,Просит реструктуризацию,28,четверг,2 days 08:09:56
2,89000350892,2024-07-09 04:11:37,2024-07-09 06:59:19,2024-07-26 20:01:48,Должник попросил перезвонить позже,30,пятница,17 days 13:02:29
3,89000350892,2024-07-09 04:11:37,2024-07-09 06:59:19,2024-07-09 04:11:37,Должник отказался платить,28,вторник,0 days 02:47:42
4,89000350892,2024-07-09 04:11:37,2024-07-09 06:59:19,2024-07-26 23:04:33,Должник попросил перезвонить позже,30,пятница,17 days 16:05:14
...,...,...,...,...,...,...,...,...
29936,89999909916,2024-07-28 20:17:02,2024-07-28 20:27:37,2024-07-28 20:17:02,Должник попросил перезвонить позже,30,воскресенье,0 days 00:10:35
29937,89999909916,2024-07-28 20:17:02,2024-07-28 20:27:37,2024-07-29 07:29:46,Платеж подтвержден,31,понедельник,0 days 11:02:09
29938,89999909916,2024-07-28 20:17:02,2024-07-28 20:27:37,2024-07-22 20:27:05,Ответ третьих лиц,30,понедельник,6 days 00:00:32
29939,89999909916,2024-07-28 20:17:02,2024-07-28 20:27:37,2024-07-23 23:23:40,Должник попросил перезвонить позже,30,вторник,4 days 21:03:57


Нахождение строки с минимальной разницей для каждой комбинации "Номер телефона" и "Дата вызова от сотрудника"

In [398]:
result_df = merged_df.loc[merged_df.groupby(['Номер телефона', 'Дата вызова от сотрудника'])['date_diff'].idxmin()]

In [399]:
result_df

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
3,89000350892,2024-07-09 04:11:37,2024-07-09 06:59:19,2024-07-09 04:11:37,Должник отказался платить,28,вторник,0 days 02:47:42
6,89000350892,2024-07-11 15:09:15,2024-07-11 16:38:49,2024-07-11 15:09:15,Просит реструктуризацию,28,четверг,0 days 01:29:34
12,89000350892,2024-07-26 20:01:48,2024-07-26 21:32:38,2024-07-26 20:01:48,Должник попросил перезвонить позже,30,пятница,0 days 01:30:50
19,89000350892,2024-07-26 23:04:33,2024-07-26 23:41:26,2024-07-26 23:04:33,Должник попросил перезвонить позже,30,пятница,0 days 00:36:53
24,89000921237,2024-07-28 15:03:46,2024-07-28 16:07:51,2024-07-28 15:03:46,Ответ третьих лиц,30,воскресенье,0 days 01:04:05
...,...,...,...,...,...,...,...,...
29904,89999429468,2024-07-19 08:31:33,2024-07-19 08:43:22,2024-07-19 08:31:33,Должник отказался платить,29,пятница,0 days 00:11:49
29915,89999429468,2024-07-27 04:18:22,2024-07-27 06:07:29,2024-07-27 04:18:22,Платеж подтвержден,30,суббота,0 days 01:49:07
29925,89999429468,2024-07-29 11:49:22,2024-07-29 13:17:18,2024-07-29 11:49:22,Платеж подтвержден,31,понедельник,0 days 01:27:56
29934,89999525670,2024-07-13 00:07:15,2024-07-13 02:58:12,2024-07-13 00:07:15,Должник отказался платить,28,суббота,0 days 02:50:57


На основе столбца "old.Дата вызова" сверим полученный результат

In [400]:
result_df[result_df['old.Дата вызова']!=result_df['Дата вызова']]

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
151,89003807051,2024-07-19 10:36:44,2024-07-19 15:44:53,2024-07-19 14:43:42,Платеж подтвержден,29,пятница,0 days 01:01:11
320,89010762026,2024-07-19 15:40:55,2024-07-19 17:22:22,2024-07-19 16:49:38,Ответ третьих лиц,29,пятница,0 days 00:32:44
574,89016116420,2024-07-17 16:22:00,2024-07-17 17:30:16,2024-07-17 17:54:19,Ответ третьих лиц,29,среда,0 days 00:24:03
674,89019591833,2024-07-24 01:02:20,2024-07-24 03:52:17,2024-07-24 02:14:10,Должник отказался платить,30,среда,0 days 01:38:07
1448,89050169062,2024-07-13 10:25:29,2024-07-13 15:23:28,2024-07-13 11:44:13,Ответ третьих лиц,28,суббота,0 days 03:39:15
...,...,...,...,...,...,...,...,...
29284,89979059903,2024-07-23 21:46:55,2024-07-23 23:07:09,2024-07-23 22:55:42,Платеж подтвержден,30,вторник,0 days 00:11:27
29562,89989023566,2024-07-22 09:53:03,2024-07-22 12:19:13,2024-07-22 09:58:27,Просит реструктуризацию,30,понедельник,0 days 02:20:46
29786,89994969058,2024-07-31 10:58:42,2024-07-31 17:26:46,2024-07-31 18:45:01,Должник отказался платить,31,среда,0 days 01:18:15
29829,89995684926,2024-07-28 09:36:45,2024-07-28 06:04:40,2024-07-28 06:37:06,Должник попросил перезвонить позже,30,воскресенье,0 days 00:32:26


106 значений подтянулись неверно. Проверим по какой причине это произошло, взяв один экземпляр для проверки

In [405]:
merged_df[merged_df['Номер телефона']==result_df[result_df['old.Дата вызова']!=result_df['Дата вызова']].iloc[0,0]]

,Номер телефона,old.Дата вызова,Дата вызова от сотрудника,Дата вызова,Результат взаимодействия,Номер недели,День недели,date_diff
151,89003807051,2024-07-19 10:36:44,2024-07-19 15:44:53,2024-07-19 14:43:42,Платеж подтвержден,29,пятница,0 days 01:01:11
152,89003807051,2024-07-19 10:36:44,2024-07-19 15:44:53,2024-07-19 10:36:44,Просит реструктуризацию,29,пятница,0 days 05:08:09
153,89003807051,2024-07-19 10:36:44,2024-07-19 15:44:53,2024-07-13 16:13:31,Платеж подтвержден,28,суббота,5 days 23:31:22
154,89003807051,2024-07-19 10:36:44,2024-07-19 15:44:53,2024-07-10 17:56:18,Платеж подтвержден,28,среда,8 days 21:48:35
155,89003807051,2024-07-19 10:36:44,2024-07-19 15:44:53,2024-07-15 11:47:25,Должник попросил перезвонить позже,29,понедельник,4 days 03:57:28


Видно, что "Дата вызова от сотрудника" находится ближе к "Дате вызова", чем "old.Дата вызова". Поэтому код сработал верно. Здесь влияние оказал часовой пояс. Обычно в таких выгрузках присутствует столбец с ФИО сотрудника, в таком случае можно из распределения специалистов взять регион, где работает сотрудник, и на основе этого региона вычислить часовой пояс.

Таким образом, удалось сопоставить наиближайшие даты из двух выгрузок

## Выявление звонков, нарушающих ФЗ-230

Напомню, ФЗ-230 дает ограничение для организаций: "телефонные звонки могут осуществляться не более одного раза в день, двух раз в неделю и восьми раз в месяц"

In [406]:
df_calls = df_calls.sort_values(by=['Номер телефона', 'Дата вызова'])

In [408]:
df_calls['Порядковый номер звонка за месяц'] = df_calls.groupby(['Номер телефона'])['Дата вызова'].rank(method='dense').astype(int)
df_calls['Порядковый номер звонка за неделю'] = df_calls.groupby(['Номер телефона', 'Номер недели'])['Дата вызова'].rank(method='dense').astype(int)
df_calls['Порядковый номер звонка за день'] = df_calls.groupby(['Номер телефона', 'День недели'])['Дата вызова'].rank(method='dense').astype(int)


In [409]:
df_calls

,Номер телефона,Дата вызова,Результат взаимодействия,Номер недели,День недели,Порядковый номер звонка за месяц,Порядковый номер звонка за неделю,Порядковый номер звонка за день
20823,89000196079,2024-07-11 02:21:19,Просит реструктуризацию,28,четверг,1,1,1
20821,89000196079,2024-07-16 03:44:26,Должник попросил перезвонить позже,29,вторник,2,1,1
20822,89000196079,2024-07-17 01:43:09,Платеж подтвержден,29,среда,3,2,1
20820,89000196079,2024-07-31 04:25:08,Ответ третьих лиц,31,среда,4,1,2
37425,89000248984,2024-07-12 23:46:58,Платеж подтвержден,28,пятница,1,1,1
...,...,...,...,...,...,...,...,...
35450,89999909916,2024-07-20 08:21:48,Должник попросил перезвонить позже,29,суббота,1,1,1
35453,89999909916,2024-07-22 20:27:05,Ответ третьих лиц,30,понедельник,2,1,1
35451,89999909916,2024-07-23 23:23:40,Должник попросил перезвонить позже,30,вторник,3,2,1
35452,89999909916,2024-07-28 20:17:02,Должник попросил перезвонить позже,30,воскресенье,4,3,1


Номера телефонов (без дубликатов), по которым было нарушение ФЗ-230 по количеству звонков за месяц:

In [431]:
pd.DataFrame({'Больше 8 звонков за месяц': df_calls[df_calls['Порядковый номер звонка за месяц']>8]['Номер телефона'].unique()})

,Больше 8 звонков за месяц
0,89000543004
1,89001392661
2,89001720780
3,89003395826
4,89003723771
...,...
688,89994427527
689,89995138909
690,89995141242
691,89996733206


Номера телефонов (без дубликатов), по которым было нарушение ФЗ-230 по количеству звонков за неделю:

In [446]:
pd.DataFrame({'Больше 2 звонков за неделю': df_calls[df_calls['Порядковый номер звонка за неделю']>2]['Номер телефона'].unique()})

,Больше 2 звонков за неделю
0,89000329976
1,89000543004
2,89000921237
3,89001121885
4,89001243766
...,...
3763,89999208519
3764,89999366080
3765,89999429468
3766,89999525670


Номера телефонов (без дубликатов), по которым было нарушение ФЗ-230 по количеству звонков за день:

In [434]:
pd.DataFrame({'Больше 1 звонка за день': df_calls[df_calls['Порядковый номер звонка за день']>1]['Номер телефона'].unique()})

,Больше 1 звонка за день
0,89000196079
1,89000329976
2,89000350892
3,89000543004
4,89000921237
...,...
7154,89999322265
7155,89999366080
7156,89999429468
7157,89999525670


Номера телефонов (без дубликатов), по которым было любое нарушение ФЗ-230:

In [435]:
dfs_list = [pd.DataFrame({'Больше 8 звонков за месяц': df_calls[df_calls['Порядковый номер звонка за месяц']>8]['Номер телефона'].unique()}).rename(columns={'Больше 8 звонков за месяц':"Номер с нарушением ФЗ-230"}),
            pd.DataFrame({'Больше 2 звонков за неделю': df_calls[df_calls['Порядковый номер звонка за неделю']>2]['Номер телефона'].unique()}).rename(columns={'Больше 2 звонков за неделю':"Номер с нарушением ФЗ-230"}),
            pd.DataFrame({'Больше 1 звонка за день': df_calls[df_calls['Порядковый номер звонка за день']>1]['Номер телефона'].unique()}).rename(columns={'Больше 1 звонка за день':"Номер с нарушением ФЗ-230"})
           ]
pd.DataFrame({'Номер с нарушением ФЗ-230': pd.concat(dfs_list,axis=0)["Номер с нарушением ФЗ-230"].unique()})

,Номер с нарушением ФЗ-230
0,89000543004
1,89001392661
2,89001720780
3,89003395826
4,89003723771
...,...
7411,89997707185
7412,89998104133
7413,89998201057
7414,89999150685


In [444]:
print(f'Напоминаю, общее количество номеров равно: {len(df_calls['Номер телефона'].unique())}')

Напоминаю, общее количество номеров равно: 10000


Таким образом, по 74% номерам совершается нарушение ФЗ-230. Можно предположить, что в таком случае программа-обзвонщик не имеет ограничения на количество звонков или работает с ошибками, поэтому некорректно выполняет эту функцию. Это необходимо исправлять.